In [ ]:
!pip install google-cloud-core==1.5.0 google-cloud-speech==2.0.1 google-cloud-storage==1.35.0 google-cloud-texttospeech==2.2.0 boto3 pydub

     |████████████████████████████████| 112kB 14.2MB/s 
     |████████████████████████████████| 102kB 5.3MB/s 
     |████████████████████████████████| 61kB 4.5MB/s 
     |████████████████████████████████| 133kB 14.1MB/s 
     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 522kB 15.2MB/s 
     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 7.5MB 20.3MB/s 
     |████████████████████████████████| 645kB 65.5MB/s 
ERROR: google-cloud-bigquery 1.21.0 has requirement google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 1.2.0 which is incompatible.
ERROR: botocore 1.20.73 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Found exi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
from pydub import AudioSegment
import moviepy.editor as mp
from google.cloud import speech_v1p1beta1 as speech
import requests
import json
from google.cloud import texttospeech
import boto3 #AMazon client library
#import configparser
import tempfile
import librosa 
from moviepy.editor import VideoFileClip, AudioFileClip
from google.cloud import storage

#config_file = configparser.ConfigParser()
#config_file.read('config.ini')

#API = str(config_file.get('google','api_name'))
#ACCESS_KEY = str(config_file.get('amazon','access_key_id'))
#SECRET_KEY = str(config_file.get('amazon','secret_access_key'))
#ACCESS_KEY = ""
#SECRET_KEY = ""

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/gdrive/MyDrive/techara/API_Key.json"
#os.environ['GOOGLE_APPLICATION_CREDENTIALS']=str('API_Key.json')


Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1753088/45929032 bytes (3.8%)4276224/45929032 bytes (9.3%)6356992/45929032 bytes (13.8%)8667136/45929032 bytes (18.9%)11444224/45929032 bytes (24.9%)13852672/45929032 bytes (30.2%)16171008/45929032 bytes (35.2%)18956288/45929032 bytes (41.3%)21471232/45929032 bytes (46.7%)24117248/45929032 bytes (52.5%)26599424/45929032 bytes (57.9%)28983296/45929032 bytes (63.1%)311

In [ ]:
def extract_original_audio_wav(originalVideoName, originalAudioName, projectDir, videoDir):
    """ Takes in the original video and extracts the audio (mono channel) in .wav format. Saves the audio to google storage.
    
    Arguments:
        originalVideoName (str): Contains the name of the original video.
        originalAudioName (str): Defines the name under which the extracted audio is going to be saved and must and with .wav
        (i.e. "audio.wav").
        projectDir (str): Name of the project folder.
        videoDir (str): Name of the folder where the video is located (i.e. "videoFiles")
    Returns:
        void : Extracts audio and saves as .wav to google storage.
    """
    try:
        #Load the video

        videoName = driveLocation + projectDir + "/" + videoDir + "/" + originalVideoName
        originalVideo = mp.VideoFileClip(videoName)
        
        #Extract the audio and save it locally
        originalVideo.audio.write_audiofile(originalAudioName, ffmpeg_params=["-ac", "1"])

        #Load the extracted audio
        with open(originalAudioName, 'rb') as original_audio_bytes:
            extractedAudio = original_audio_bytes.read()
            
        print("Audio successfully extracted and saved as " + originalAudioName)
        print("Proceeding to upload to storage")
        storage_client = storage.Client()
        bucket = storage_client.bucket("syntechara-bucket")
        blob = bucket.blob(originalAudioName)

        blob.upload_from_filename(originalAudioName, content_type="audio/wav")
        print("Extracted audio successfully uploaded to storage.")    
            
            
    except Exception as e:
        print(e)
        print("Something went wrong while extracting audio.")
        

In [ ]:
def parse_google_stt(googleSpeechToTextResult):
    """ Parses the result obtained from the STT engine into a more usable format.
    
    Arguments:
        googleSpeechToTextResult: Result obtained from the STT engine after transcription
    
    Returns:
        parsedData: The result returned contains the entire transcription, and every single word with its start time, end time
        and speaker (if possible)
        [{'transcript': 'meine letzten Zeit ...', 
         'words': [{'word': 'meine','start_time': 0.0,'end_time': 0.3,'speaker_tag': 0},
                   {'word': 'letzten', 'start_time': 0.3, 'end_time': 0.6, 'speaker_tag': 0}
                    ...]}]
    """
    json = []
    for section in googleSpeechToTextResult.results:
        data = {
            "transcript": section.alternatives[0].transcript,
            "words": []
        }
        for word in section.alternatives[0].words:
            data["words"].append({
                "word": word.word,
                "start_time": word.start_time.total_seconds(),
                "end_time": word.end_time.total_seconds(),
                "speaker_tag": word.speaker_tag
            })
        print(data)
        json.append(data)
    return json
        

In [ ]:
def parse_single_speaker_sentences(parsedData, originalLanguage, pauseLength = 1):
    """ Takes parsedData from parse_google_stt() and splits the sentences according to the speaker
    
    Arguments: 
        parsedData str[]: Output from parse_google_stt() 
        [{"transcript": "Hello there my example", "words": [{"word": "Hello", "start_time": 1, "end_time": 2, "speaker_tag: 1}]}]
        originalLanguage str: Defines the language code of the original video (i.e. "de-DE"). List of all supported languages
        can be found at https://cloud.google.com/speech-to-text/docs/languages.
        pauseLength float: If time between words bigger than pauseLength, we consider it a new sentence. Default value is 1s.
    Returns:
        parsedSentences str[]: [{"sentence": "Hello there", "speaker": 1, "start_time": 1, "end_time": 2.5}]
    
    """

    
    def get_word(word, originalLanguage):
        if originalLanguage == "ja":
            return word.split('|')[0]
        return word

    sentences = []
    sentence = {}
    for result in parsedData:
        for i, word in enumerate(result["words"]):
            wordText = get_word(word["word"], originalLanguage)
            if not sentence:
                sentence = {
                    originalLanguage: [wordText],
                    "speaker": word["speaker_tag"],
                    "start_time": word["start_time"],
                    "end_time": word["end_time"]
                }
            # If we have a new speaker, save the sentence and create a new one:
            elif word["speaker_tag"] != sentence["speaker"]:
                sentence[originalLanguage] = ' '.join(sentence[originalLanguage])
                sentences.append(sentence)
                sentence = {
                    originalLanguage: [wordText],
                    "speaker": word["speaker_tag"],
                    "start_time": word["start_time"],
                    "end_time": word["end_time"]
                }
            # If the sentence is ended with a dot, save the sentence and create a new one:
            elif i+1 < len(result["words"]) and result["words"][i-1]["word"][-1] == ".":
                sentence[originalLanguage] = " ".join(sentence[originalLanguage])
                sentences.append(sentence)
                sentence = {
                    originalLanguage: [wordText],
                    "speaker": word["speaker_tag"],
                    "start_time": word["start_time"],
                    "end_time": word["end_time"]
                }
            else:
                sentence[originalLanguage].append(wordText)
                sentence["end_time"] = word["end_time"]

            if i+1 < len(result["words"]):
                # Check the amount of time between the end of the current and start of the next word
                gapBetweenWords = float(result["words"][i+1]["start_time"]) - float(word["end_time"])
            
                # If greater than the defined pauseLength, treat it as a new sentence
            if gapBetweenWords > pauseLength:
                
                parsedSentence["text"] = " ".join(parsedSentence["text"])
                parsedSentences.append(parsedSentence)
                parsedSentence = {}
                
        if sentence:
            sentence[originalLanguage] = ' '.join(sentence[originalLanguage])
            sentences.append(sentence)
            sentence = {}

    return sentences

In [ ]:
def transcribe(originalAudioName, originalLanguage, projectDir, speechContexts=[], speakerCount=1):
    """ Transcribes the original audio and saves the transcription to a .txt file
    
    Arguments:
        originalAudioName (str): Name under which the extracted audio was saved.
        originalLanguage (str): Defines the language code of the original video (i.e. "de-DE"). List of all supported languages
        can be found at https://cloud.google.com/speech-to-text/docs/languages.
        projectDir (str): Name of the project folder.
        speechContexts (str[]): Defines uncommon words that are very likely to appear in the speech. If not entered, takes the
        default value as null.
        speakerCount (int, optional): Defines the number of speakers and works only in English. If not entered takes the default
        value null.
        
    Returns:
        void: Writes the transcription to a .txt file
        
    """    
    try:
        
        diarizationConfig = speech.SpeakerDiarizationConfig(
        enable_speaker_diarization=speakerCount if speakerCount > 1 else False,
        )
        
        gcs_path = gcsPath + originalAudioName

        audio = speech.RecognitionAudio(uri=gcs_path)
        
        config = speech.RecognitionConfig(
                language_code=originalLanguage ,
                enable_automatic_punctuation=True,
                enable_word_time_offsets=True,
                speech_contexts=[],
                diarization_config=diarizationConfig,
                profanity_filter=True,
        
            )
        
        client = speech.SpeechClient()
        
        response = client.long_running_recognize(config=config, audio=audio)
             
        parsedData = parse_google_stt(response.result())
            
        parsedSentences = parse_single_speaker_sentences(parsedData, originalLanguage)
        
        transcribedTextFileLocation = driveLocation + projectDir + "/" + originalAudioName[:-4] + "_transcription.txt"
        
        json.dump(parsedSentences, open((transcribedTextFileLocation), "w"))

        print("Transcription saved to " + transcribedTextFileLocation )
    
    except Exception as e:
        print("Something went wrong while transcribing.")
        print(e)
        

In [ ]:
def translate(originalAudioName, transcriptionFileName, projectDir, targetLanguageDeepl, sourceLanguageDeepl):
    """ Translates the content of the transcriptionFileName file from the original language to the target language, then
        saves the translation in (transcriptionFileName + _translation.txt)
        
    Arguments:
        originalAudioName (str): Name under which the extracted audio was saved.
        transcriptionFileName (str): Name under which the transcribed speech was saved.
        projectDir (str): Name of the project folder.
        sourceLanguageDeepl (str): Defines the language code of the original video ("DE") used for transcription. List of 
        all supported languages can be found at https://www.deepl.com/docs-api/translating-text/request/ under
        source_lang.
        targetLanguageDeepl (str): Defines the target language code. More at https://www.deepl.com/docs-api/translating-text/request/ under
        target_lang.

        
    Returns:
        void: Saves the translation to (originalAudioName_translation.txt)
    """
    try:
        transcriptionFileLocationName = driveLocation + projectDir + "/" + transcriptionFileName
        with open(transcriptionFileLocationName, "r") as text_file:
            checkedTranscription = json.loads(text_file.read())
        
        translatedSentences = []
        translatedSentence = {}
        
        for i, sentence in enumerate(checkedTranscription):
            
            r =  requests.post(url='https://api-free.deepl.com/v2/translate',
                            data = {
                                'source_lang' : sourceLanguageDeepl,
                                'target_lang' : targetLanguageDeepl,  
                                'auth_key' : '##################',
                                'text': sentence[originalLanguage]
                            })
        
            translatedJson = json.loads(r.text)
        
            translatedText = translatedJson["translations"][0]['text']
            print(translatedText)
            
            translatedSentence = {
                    targetLanguageDeepl: translatedText,
                    "speaker": sentence["speaker"],
                    "start_time": sentence["start_time"],
                    "end_time": sentence["end_time"]
                }
            
            translatedSentences.append(translatedSentence)
            translatedSentence = {}

        translatedFileName = driveLocation + projectDir + "/" + originalAudioName[:-4] + "_translation.txt"

        json.dump(translatedSentences, open((translatedFileName), "w"))
        print("Translated file saved to " + translatedFileName)
    except Exception as e:
        print("Something went wrong while translating.")
        print(e)

 

In [ ]:
def text_to_ssml(inputText):
    """Adds the <speak></speak> tags to the text to be synthesized. Replaces reserved characters (i.e. ' -> &apos;)
    Arguments:
        inputText (str): The text to which the opening and closing ssml tags will be added.
        
    Returns:
        inputText (str): Text with opening and closing ssml tags added and reserved characters changed."""
    
    try:
        inputText = (inputText.replace("&", "&amp;")).replace("<", "&lt;").replace(">", "&gt;").replace('"', '&quot;').replace("'", '&apos;')
        inputText = "<speak>" + inputText + "</speak>"
        return inputText
    except Exception as e:
        print(e)
        print("Something went wrong while adding SSML tags.")

In [ ]:
def synthesize_speech_google(text, targetLanguage, targetAudioName, projectDir, audioDir, voiceName=None, speechSpeed=1):
    """ Takes in the checked translated text and synthesizes speech in the target language. Saves 
        the speech as .mp3.
        
    Arguments:
        text (str): Text to be synthesized.
        targetLanguage (str): Defines the language code of the target audio.
        targetAudioName (str): Name under which the audio will be saved.
        outputDirectoryName (str): Name of the folder where the audio will be saved.
        voiceName (str, optional): You can define the preffered voice to be used. See more 
        at https://cloud.google.com/text-to-speech/docs/voices
        speechSpeed (int): Defaults to 1.
       
        
        
    Returns:
        void: Saves the synthesized speech as .wav or mp3
    
    """
    try:
            
        #Initialize speech to text client
        client = texttospeech.TextToSpeechClient()
        
        ssml_input_text = text_to_ssml(text)

        #Voice settings
        voice = texttospeech.VoiceSelectionParams(
                    language_code=targetLanguage, name=voiceName
                )

        synthesis_ssml_input = texttospeech.SynthesisInput(ssml=ssml_input_text)
        
        audio_config = texttospeech.AudioConfig(
                audio_encoding=texttospeech.AudioEncoding.MP3,
                speaking_rate=speechSpeed
            )

        response = client.synthesize_speech(
                input=synthesis_ssml_input,
                voice = voice,
                audio_config=audio_config
            )
        

        speechFileLocation = driveLocation + projectDir + "/" + audioDir + "/" + targetAudioName

        
        with open(speechFileLocation, "wb") as out:
            # Write the response to the output file.
            out.write(response.audio_content)
            print('Synthesized speech written as ' + speechFileLocation)
            
    except Exception as e:
        print("Something went wrong while synthesizing the speech.")
        print(e)

        
        
         

In [ ]:
def synthesize_timelimited_speech_google(translationFileName, projectDir, targetLanguageDeepl,targetLanguage, audioDir,voiceName=None):
    """Synthesizes speech within a certain time limit using Google TTS.
    
    Arguments:
        originalText (str): Text to be synthesized.
        targetLanguage (str): Defines the language code of the target audio. See more at https://cloud.google.com/text-to-speech/docs/voices
        speechDuration (float): Time limit in seconds.
        outputDirectoryName (str): Name of the folder where the audio will be saved.
        voiceName (str, optional): Voice to be used.
          
        
    Returns:
        void: Saves speech in .wav format.
    
    """
    with open(driveLocation + projectDir + "/" + translationFileName, "r") as text_file:
            checkedTranslation = json.loads(text_file.read())
    
    for i, sentence in enumerate(checkedTranslation):
        audioName = str(i) + ".wav"
        speechDuration = sentence['end_time'] - sentence['start_time']
        synthesizedAudioName = driveLocation + projectDir + "/" + audioDir + "/" + audioName
        
        baseAudio = synthesize_speech_google(sentence[targetLanguageDeepl], targetLanguage, audioName, projectDir, audioDir, voiceName, speechSpeed=1)

        y, sr = librosa.load(synthesizedAudioName)
        baseDuration = librosa.get_duration(y=y, sr=sr)
    
        ratio = baseDuration / speechDuration
    
        if ratio <= 0.8:
            ratio = 0.8
        
        elif ratio > 2:
            ratio = 2
      
        synthesize_speech_google(sentence[targetLanguageDeepl],targetLanguage,audioName,projectDir, audioDir, voiceName, ratio)
    

In [ ]:
def synthesize_speech_polly(text,regionName, defaultUrl, accessKey, secretAccessKey, voiceNamePolly):
    """ AWS Polly TTS to synthesize humanlike voices. 
    
    Arguments:
        regionName (str): Defaults to 'us-east-2', should not be changed unless the AWS account changes.
        defaultUrl (str): Url where request for speech synthesis are made, should not be changed unless the AWS account changes.
        accessKey (str): Access key for the AWS services, loaded from the .csv file.
        secretAccessKey(str): Same as accessKey.
        voiceNamePolly (str): Voice to be used during the speech synthesis. More at https://docs.aws.amazon.com/polly/latest/dg/voicelist.html 
    """
    try:
        polly = boto3.client('polly', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY, region_name=regionName)
    
        #Load the translated text
        #with open("translation.txt", "r") as text_file:
            #checkedTranslation = text_file.read() 
    
        resp = polly.synthesize_speech(OutputFormat='mp3', Text=text, VoiceId=voiceNamePolly)
    
        soundBytes = resp['AudioStream'].read()
    
        with open(targetAudioNamePolly, "wb") as out:
                # Write the response to the output file.
                out.write(soundBytes)
                print('Synthesized speech written as ' + targetAudioNamePolly)
                
    except Exception as e:
        print("Something went wrong while synthesizing the speech with Polly.")
        print(e)

In [ ]:
#TODO: Figure out a way to solve word placing at proper time
def synthesize_timelimited_speech_polly(text, regionName, defaultUrl, accessKey, secretAccessKey, voiceNamePolly, speechDuration):
    try:
        synthesize_speech_polly(text, regionName, defaultUrl, accessKey, secretAccessKey, voiceNamePolly)
        print("First speech synthesized")
        y, sr = librosa.load(targetAudioNamePolly)
        print("librosa loaded")
        baseDuration = librosa.get_duration(y=y, sr=sr)
        
        print("base duration is", baseDuration)
        
        ratio = baseDuration / speechDuration
        print("ratio is", ratio)   
    
    
        if ratio <= 0.8:
            ratio = 0.8
        
        elif ratio > 2:
            ratio = 2
        
        sy
        
        
    except Exception as e:
            print("Something went wrong while synthesizing timelimited speech with Polly")

In [ ]:
def stitch_audio_video_google(originalVideoName, translationFileName, projectDir, audioDir, videoDir,outFileDir, overlayGain):
    """ Combines properly placed audio with the video file and creates the dubbed video
    
    Arguments:
        parsedSentences (list): Sentences parsed using parse_single_speaker_sentences function.
        audioDir (str): Directory containing the synthesized speech audios.
        videoDir (str): Directory containing the video file.
        outFileDir (str): Directory that will contain the dubbed video.
        
    Returns:
        void: Writes the dubbed video to the outFileDir location.
    """
    with open(driveLocation + projectDir + "/" + translationFileName, "r") as text_file:
            checkedTranslations = json.loads(text_file.read())
    audioFiles = os.listdir(driveLocation + projectDir + "/" + audioDir)
    audioFiles.sort(key=lambda x: int(x.split('.')[0]))
    
    movieFile = driveLocation + projectDir + "/" + videoDir + "/" + originalVideoName
    segments = [AudioSegment.from_file(
        os.path.join(driveLocation + projectDir + "/" + audioDir, x)) for x in audioFiles]
    # Also, grab the original audio
    dubbed = AudioSegment.from_file(movieFile)
    
    # Place each computer-generated audio at the correct timestamp
    for sentence, segment in zip(checkedTranslations, segments):
        dubbed = dubbed.overlay(
            segment, position=sentence['start_time'] * 1000, gain_during_overlay=overlayGain)
    # Write the final audio to a temporary output file
    audioFile = tempfile.NamedTemporaryFile()
    dubbed.export(audioFile)
    audioFile.flush()
    
    clip = VideoFileClip(movieFile)
    audio = AudioFileClip(audioFile.name)
    clip = clip.set_audio(audio)

    dubbedVideoName = originalVideoName[:-4] + "_dubbed" + originalVideoName[-4:]
    clip.write_videofile(driveLocation + projectDir + "/" + videoDir + "/" + dubbedVideoName, codec='libx264', audio_codec='aac')
    audioFile.close()

In [ ]:
"""
    Required Inputs:
    driveLocation (str): Contains the name of the main folder and its location in your google drive.
    gcsPath (str): Path to the google cloud storage bucket where the extracated audio will be saved.

    originalVideoName (str): Contains the name of the original video.
    originalAudioName (str): Defines the name under which the extracted audio is going to be saved and must and with .wav
    (i.e. "audio.wav").

    originalLanguage (str): Defines the language code of the original video (i.e. "de-DE") used for transcription. List of 
    all supported languages can be found at https://cloud.google.com/speech-to-text/docs/languages .
    targetLanguage (str): Defines the language code of the target audio.

    sourceLanguageDeepl (str): Defines the language code of the source language for the DeepL translation API.
    targetLanguageDeepl (str): Defines the language code of the target language for the DeepL translation API.

    projectDir (str): Name of the directory where each separate dubbing project will be saved.
    videoDir (str): Name of the directory where videos related to the project will be stored.
    audioDir (str): Name of the directory where the generated audio files will be stored.

    speakerCount (int, optional): Defines the number of speakers and works only in English. If not entered takes the default
    value 1.
    overlayGain (int): Defines how much the original audio will be silenced in comparison to the newly generated audio.
    voiceNameGoogle (str): Defines the voice to be used for speech synthesis.
    pauseLength (float): The amount of time between 2 consecutive words that has to pass in order to start a new sentence.

    targetAudioName(str): Defines the name under which the synthesized speech will be saved.
    targetVideoName (str):Defines the name under which the target video will be saved.
    voiceNamePolly (str): Defines the voice to be used in Polly, more at https://docs.aws.amazon.com/polly/latest/dg/voicelist.html

    transcriptionFileName (str): Name of the .txt file where the transcription is saved. It should be in the form 
    videoName + transcription (hashoshi_transcription.txt)
    translationFileName (str): Name of the .txt file where the translation is saved. It should be in the form 
    videoName + translation (hashoshi_translation.txt)
"""

driveLocation = "/content/gdrive/MyDrive/techara/"
gcsPath = "gs://syntechara-bucket/" 

originalVideoName = "hashoshi.mp4"
originalAudioName = "hashoshi.wav"

originalLanguage = "en-US"
targetLanguage = "fr-FR"

sourceLanguageDeepl = "EN"
targetLanguageDeepl = "FR"

projectDir = "hashoshi"
videoDir = "videoFiles"
audioDir = "audioFiles"
outFileDir = "videoFiles"

speakerCount = 1
overlayGain = -30
voiceNameGoogle = "fr-FR-Wavenet-B"
pauseLength = 0.5

transcriptionFileName = "hashoshi_transcription.txt"
translationFileName = "hashoshi_translation.txt"

#targetAudioNameWav = "target_audio.wav"
#targetAudioNameMp3 = "target_audio.mp3"
#targetAudioNamePolly = "target_audio.mp3"
#targetVideoName = "target_video.mp4"
#voiceNamePolly = "Brian"

#AWS Polly

#defaultUrl = 'https://polly.us-east-2.amazonaws.com'
#regionName = 'us-east-2'

In [ ]:
#extract_original_audio_wav("hosk_3min.mp4", "hosk_3min.wav", "hosk", "videoFiles")
extract_original_audio_wav(originalVideoName, originalAudioName, projectDir, videoDir)

[MoviePy] Writing audio in hashoshi.wav


100%|██████████| 3840/3840 [00:01<00:00, 2093.55it/s]

[MoviePy] Done.


Audio successfully extracted and saved as hashoshi.wav
Proceeding to upload to storage
Extracted audio successfully uploaded to storage.


In [ ]:
#transcribe("hosk_3min.wav", "en-US", "hosk", speechContexts=[], speakerCount=1)
transcribe(originalAudioName, originalLanguage, projectDir, speechContexts=[], speakerCount=1)

{'transcript': "Ladies and Gentlemen, people of the internet. Today, I want to share with you the best mobile wallet for cryptocurrency, that will help you keep your daily crypto usage, safe and secure. And by the way, if you're into cryptocurrency in blockchain, please do consider subscribing to the channel and hitting that little bell. Notification button below so you can get a heads up whenever I post new content. Thanks in advance for the further Ado. Let's hash it out.", 'words': [{'word': 'Ladies', 'start_time': 0.0, 'end_time': 0.5, 'speaker_tag': 0}, {'word': 'and', 'start_time': 0.5, 'end_time': 0.6, 'speaker_tag': 0}, {'word': 'Gentlemen,', 'start_time': 0.6, 'end_time': 1.0, 'speaker_tag': 0}, {'word': 'people', 'start_time': 1.0, 'end_time': 1.3, 'speaker_tag': 0}, {'word': 'of', 'start_time': 1.3, 'end_time': 1.4, 'speaker_tag': 0}, {'word': 'the', 'start_time': 1.4, 'end_time': 1.5, 'speaker_tag': 0}, {'word': 'internet.', 'start_time': 1.5, 'end_time': 1.8, 'speaker_tag'

In [ ]:
#translate("hosk_3min.wav", "hosk_3min_transcription.txt", "hosk", "IT", "EN")
translate(originalAudioName, transcriptionFileName, projectDir, targetLanguageDeepl, sourceLanguageDeepl)

Mesdames et messieurs, gens d'Internet.
Aujourd'hui, je veux partager avec vous le meilleur portefeuille mobile pour les crypto-monnaies, qui vous aidera à garder votre utilisation quotidienne de crypto-monnaies, sûre et sécurisée.
Et d'ailleurs, si vous aimez les crypto-monnaies et la blockchain, pensez à vous abonner à la chaîne et à appuyer sur la petite cloche.
Le bouton de notification ci-dessous vous permet d'être averti dès que je publie un nouveau contenu.
Merci d'avance pour le complÃ©ment d'information.
Discutons-en.
Le sujet de la vidéo d'aujourd'hui est mon porte-monnaie mobile préféré, en particulier pour iOS, et il s'agit de toutes les pièces de monnaie.
Coin est une application de portefeuille de crypto-monnaies bien conçue, disponible sur IOS et Android, qui dispose de fonctions d'échange intégrées et constitue une excellente option pour l'utilisation quotidienne des crypto-monnaies.
L'interface est super propre.
Il a tout ce dont vous avez besoin directement sur la pag

In [ ]:
#synthesize_timelimited_speech_google("hosk_3min_translation.txt", "hosk", "IT","it-IT", "audioFiles", voiceName="it-IT-Wavenet-D")
synthesize_timelimited_speech_google(translationFileName, projectDir, targetLanguageDeepl,targetLanguage, audioDir,voiceNameGoogle)#

Synthesized speech written as /content/gdrive/MyDrive/techara/hashoshi/audioFiles/0.wav
Synthesized speech written as /content/gdrive/MyDrive/techara/hashoshi/audioFiles/0.wav
Synthesized speech written as /content/gdrive/MyDrive/techara/hashoshi/audioFiles/1.wav
Synthesized speech written as /content/gdrive/MyDrive/techara/hashoshi/audioFiles/1.wav
Synthesized speech written as /content/gdrive/MyDrive/techara/hashoshi/audioFiles/2.wav
Synthesized speech written as /content/gdrive/MyDrive/techara/hashoshi/audioFiles/2.wav
Synthesized speech written as /content/gdrive/MyDrive/techara/hashoshi/audioFiles/3.wav
Synthesized speech written as /content/gdrive/MyDrive/techara/hashoshi/audioFiles/3.wav
Synthesized speech written as /content/gdrive/MyDrive/techara/hashoshi/audioFiles/4.wav
Synthesized speech written as /content/gdrive/MyDrive/techara/hashoshi/audioFiles/4.wav
Synthesized speech written as /content/gdrive/MyDrive/techara/hashoshi/audioFiles/5.wav
Synthesized speech written as /c

In [ ]:
#stitch_audio_video_google("hosk_3min.mp4","hosk_3min_translation.txt", "hosk","audioFiles","videoFiles", "videoFiles", -50)
stitch_audio_video_google(originalVideoName, translationFileName, projectDir, audioDir, videoDir,outFileDir, overlayGain)

[MoviePy] >>>> Building video /content/gdrive/MyDrive/techara/hashoshi/videoFiles/hashoshi_dubbed.mp4
[MoviePy] Writing audio in hashoshi_dubbedTEMP_MPY_wvf_snd.mp4


100%|██████████| 3841/3841 [00:11<00:00, 342.77it/s]


[MoviePy] Done.
[MoviePy] Writing video /content/gdrive/MyDrive/techara/hashoshi/videoFiles/hashoshi_dubbed.mp4


100%|██████████| 5224/5224 [05:09<00:00, 16.91it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/gdrive/MyDrive/techara/hashoshi/videoFiles/hashoshi_dubbed.mp4 

